In [ ]:
import requests
import csv

# training data
url = "http://kevincrook.com/utd/market_basket_training.txt"
req = requests.get(url)
#req.content
train =  open('mkt_bkt_train.txt', 'wb')
train.write(req.content)
train.close()

# testing data
url_new = "http://kevincrook.com/utd/market_basket_test.txt"
req_new = requests.get(url_new)
#req_new.content
test =  open('mkt_bkt_test.txt', 'wb')
test.write(req_new.content)
test.close()

row = []
with open('mkt_bkt_train.txt','rt', encoding = 'utf-8') as fl:
    reader = csv.reader(fl, delimiter = ',')
    for ln in reader:
        row.append(ln[1:])
fst_row = row[0]
new_row = [fst_row]

for i in row:
    put = True
    for j in new_row:
        if set(i) == set(j):
            put = False
    if (put):
        new_row.append(i)
#print(new_row)

freq = []
for new_rec in new_row:
    count = 0
    for x in row:
        if set(new_rec) == set (x):
            count += 1    
    freq.append([new_rec,count])
freq = sorted(freq,key = lambda x:x[1],reverse = True)
sortitems = []
for x in freq:
    sortitems.extend(x[:-1])

def division(data,leng):
    listnew = []
    for x in data:
        if len(x) == leng:
            listnew.append(x)
    return(listnew)
fourfreItems = division(sortitems,4)
threefreItems = division(sortitems,3)
twofreItems = division(sortitems,2)
#print(twofreItems)

test_data = []
with open('mkt_bkt_test.txt','rt', encoding = 'utf-8') as fl:
    reader = csv.reader(fl, delimiter = ',')
    for ln in reader:
        test_data.append(ln[1:])

def recommd(data,listsearch):
    for x in listsearch:
        if set(data).issubset(x):
                recom = [set(x) - set(data)]
                break
    else:
        recom = 'None'
    return(recom)

final_recommendation = []
for rec in test_data:
    data = [i for i in rec if ((i != 'P04')and (i != 'P08'))]
    #print(data)
    if len(data) == 1:
         result = recommd(data,twofreItems)
    elif len(data) == 2:
         result = recommd(data,threefreItems)
    elif len(data) == 3:
         result = recommd(data,fourfreItems)
    
    final_recommendation.append(result)
#print(final_recommendation)

index = ["{0:03}".format(i) for i in range(1,101)]
with open('market_basket_recommendations.txt', mode='wt') as opt_fl: 
        for i in range(0,100):
            if i<99:
                ln = str(index[i]) + ',' + str(final_recommendation[i][0].pop()) +'\n'
            else:
                ln = str(index[i]) + ',' + str(final_recommendation[i][0].pop())
                print(ln)
            opt_fl.write(ln)